In [ ]:
# import numpy as np
# import re
# import pandas as pd
# import geopandas as gpd
# import gdown
import ee
import google
# import os
# import geemap

# from tobler.util import h3fy
# from ee_jupyter.ipyleaflet import Map

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
ee.Authenticate()
ee.Initialize()

Seems like climate scientists love their 30 year averages. Go ahead and compare 1980-2010 with 2010-present ?
What about 2000-2010 and 2010-2020?

In [ ]:
daily = ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR")

In [ ]:
daily1980_2010 = daily.filterDate('1980-01-01', '2009-12-31').select('temperature_2m').mean()

In [ ]:
daily2010_now = daily.filterDate('2010-01-01', '2023-12-31').select('temperature_2m').mean()

In [ ]:
counties = ee.FeatureCollection("TIGER/2018/Counties")

In [ ]:
ak = counties.filter(ee.Filter.eq('STATEFP', '02'))

In [ ]:
ak.size().getInfo()

In [ ]:
prewarming = daily1980_2010.reduceRegions(collection=ak, reducer=ee.Reducer.mean(), scale=11132)
prewarming = prewarming.select(['NAMELSAD', 'mean'])
prewarming

In [ ]:
postwarming = daily2010_now.reduceRegions(collection=ak, reducer=ee.Reducer.mean(), scale=11132)
postwarming = postwarming.select(['NAMELSAD', 'mean'])
postwarming

In [ ]:
pre_boroughs = []
pre_temperature = []
preL = prewarming.toList(30)

for i in range(prewarming.size().getInfo()):
    pre_temperature.append(ee.Feature(preL.get(i)).get('mean').getInfo())
    pre_boroughs.append(ee.Feature(preL.get(i)).get('NAMELSAD').getInfo())

pre_temperature, pre_boroughs

In [ ]:
post_boroughs = []
post_temperature = []
postL = postwarming.toList(30)

for i in range(postwarming.size().getInfo()):
    post_temperature.append(ee.Feature(postL.get(i)).get('mean').getInfo())
    post_boroughs.append(ee.Feature(postL.get(i)).get('NAMELSAD').getInfo())

In [ ]:
pre_boroughs == post_boroughs

In [ ]:
pre_temperature_f = [(temp - 273.15) * 1.8 + 32 for temp in pre_temperature]

In [ ]:
post_temperature_f = [(temp - 273.15) * 1.8 + 32 for temp in post_temperature]

In [ ]:
difference = [post - pre for post, pre in zip(post_temperature_f, pre_temperature_f)]

In [ ]:
difference_greater2 = [temp > 2 for temp in difference]

In [ ]:
print([b for a, b in zip(difference_greater2, pre_boroughs) if a])

In [ ]:
difference

In [ ]:
sum(difference) / len(difference)